In [30]:
import pandas as pd
import numpy as np 
import os
from handle_dataset.transform import create_df_with_datetimes
from ml_models.lightgbm import lightgbm


In [31]:
from evaluation_protocol.grubbs import grubbs_score
from evaluation_protocol.mape import mape
from evaluation_protocol.smape import smape
from evaluation_protocol.shape_similarity import dtw

In [32]:
#Dataset
train_df = pd.read_csv('Dataset/Yearly-train.csv')
test_df = pd.read_csv('Dataset/Yearly-test.csv')

In [33]:
# Testing the new date thing
for counter in range(2):
    datetime_df = create_df_with_datetimes(train_df, counter)

# The above works!
# It will create a new dataframe with two columns 'datetime' and 'target' for as many times as you put in range 

In [34]:
# Loading just the first timeseries 
first_timeseries_train = create_df_with_datetimes(train_df, 0)
first_timeseries_test = create_df_with_datetimes(test_df, 0)

In [36]:
#Traditional models
from traditional_models.arima import arima_model
from traditional_models.theta_model import theta_model_forecast

arima_forecasts = arima_model(series=first_timeseries_train['target'].astype(float), forecast_periods=6)
theta_forecasts = theta_model_forecast(series=first_timeseries_train['target'], h=6)

/home/george/.local/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/george/.local/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/george/.local/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/george/.local/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/home/george/.local/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is avai

In [37]:
#Machine Learning models
from ml_models.prophet_model import prophet_model

#LightGBM
# for counter in range(3):
#     timeseries_df = create_df_with_datetimes(train_df, counter)
#     lightgbm(timeseries_df)

# Prophet
prophet_forecasts = prophet_model(train=first_timeseries_train, test=first_timeseries_test)


/home/george/.local/lib/python3.8/site-packages/prophet/forecaster.py:1124: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  self.history_dates = pd.to_datetime(pd.Series(df['ds'].unique(), name='ds')).sort_values()
/home/george/.local/lib/python3.8/site-packages/prophet/forecaster.py:271: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['ds'] = pd.to_datetime(df['ds'])
19:01:27 - cmdstanpy - INFO - Chain [1] start processing


19:01:27 - cmdstanpy - INFO - Chain [1] done processing


In [42]:
# Evaluation Protocol

alpha = 0.05
predicted_dictionary = {"arima": arima_forecasts,
                        "theta": theta_forecasts,
                        "prophet": prophet_forecasts}


# For some reason the column names change to ds and y and cannot change back....
real = first_timeseries_test['y']

for key in predicted_dictionary:
    predicted = predicted_dictionary[key]
    grubbs_test_score = grubbs_score(predicted, real, alpha)
    smape_score = smape(real.tolist(), predicted.tolist())
    shape_similarity_score = dtw(predicted, real)
    mape_score = mape(real.tolist(), predicted.tolist())

    print(f"Grubbs score for {key} is : {grubbs_test_score}")
    print(f"SMAPE score for {key} is : {smape_score}")
    print(f"MAPE score for {key} is : {mape_score}")
    print(f"Shape similarity score for {key} is : {shape_similarity_score}")
    print("===================================================================================")

print('\n')
print('\n')

# Printing the new string:
scores_dict = {
    'arima': [],
    'theta': [],
    'prophet': []
}

for key in predicted_dictionary:
    predicted = predicted_dictionary[key]
    grubbs_test_score = grubbs_score(predicted, real, alpha)
    smape_score = smape(real.tolist(), predicted.tolist())
    shape_similarity_score = dtw(predicted, real)
    mape_score = mape(real.tolist(), predicted.tolist())

    # MAPE | sMAPE | Grubbs | tShape similarity
    scores_dict[key].append(round(mape_score,2))
    scores_dict[key].append(round(smape_score,2))
    scores_dict[key].append(round(grubbs_test_score))
    scores_dict[key].append(round(shape_similarity_score))

from evaluation_protocol.evaluation_protocol_string import eval_string

evaluation_string = eval_string(scores_dict)
print(evaluation_string)

Grubbs score for arima is : 0
SMAPE score for arima is : 9.64446964462614
MAPE score for arima is : 9.121537475242345
Shape similarity score for arima is : 4178.792879752498
Grubbs score for theta is : 0
SMAPE score for theta is : 40.00131259796595
MAPE score for theta is : 33.29745638084341
Shape similarity score for theta is : 15127.100000000006
Grubbs score for prophet is : 0
SMAPE score for prophet is : 1.9794528840151553
MAPE score for prophet is : 2.0084528351210076
Shape similarity score for prophet is : 780.8762437136947




Model		|	MAPE	|	sMAPE	|	Grubbs	|	Shape similarity	|
--------------------------------------------------------------------------------------------------
arima		|	9.12	|	9.64	|	0	|	4179			|
--------------------------------------------------------------------------------------------------
theta		|	33.3	|	40.0	|	0	|	15127			|
--------------------------------------------------------------------------------------------------
prophet		|	2.01	|	1.98	|	0	|	781			|
--